In [ ]:
import pandas as pd

# Assuming `df` is your DataFrame with the data

# Convert 'Day' to datetime
df['Day'] = pd.to_datetime(df['Day'], format='%d/%m/%Y')

# Sort the data by Patient ID and Day
df = df.sort_values(by=['Patient ID', 'Day'])

# Ensure numerical features are in the right format (e.g., floats for continuous variables)
continuous_columns = ['Resting heart rate', 'Active heart rate', 'HRV', 'Sleep duration', 
                      'Respiratory rate', 'Body temperature', 'Blood oxygen saturation', 
                      'Activity levels', 'Calories burnt', 'Weight gain (kg)']

df[continuous_columns] = df[continuous_columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
from timegan import TimeGAN

# Prepare your data (make sure it's in the shape (num_samples, time_steps, num_features))
# TimeGAN expects input data as 3D arrays: (samples, time_steps, features)
time_steps = 10  # For example, 10 days of data as a sequence
num_features = len(continuous_columns)  # Number of features for each time step

# Example: Let's convert the data into time series windows (adjust time_steps as needed)
def create_time_series_windows(df, time_steps):
    data = []
    for patient_id in df['Patient ID'].unique():
        patient_data = df[df['Patient ID'] == patient_id][continuous_columns].values
        for i in range(len(patient_data) - time_steps):
            data.append(patient_data[i:i + time_steps])
    return np.array(data)

time_series_data = create_time_series_windows(df, time_steps)

# Initialize and train the TimeGAN model
model = TimeGAN(time_steps=time_steps, features=num_features)
model.fit(time_series_data)  # AI learns the temporal dependencies and features

# Generate synthetic time series data
synthetic_data = model.sample(1000)  # Generate 1000 synthetic samples
